<h1>GLI3: Demurger et al 2015</h1>
<p>Extract the clinical data from <a href="https://pubmed.ncbi.nlm.nih.gov/24736735/"target="__blank">Démurger F, et al. (2015) New insights into genotype-phenotype correlation for GLI3 mutations. Eur J Hum Genet ;23(1):92-102. PMID:24736735</a>.<p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
import math
from csv import DictReader
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import re
from pyphetools.creation import *
from pyphetools.output import PhenopacketTable
# last tested with pyphetools version 0.3.2

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)
pmid="PMID:29198722"

In [43]:
df1 = pd.read_csv("input/demurger_table_1.csv", delimiter="\t")
df1.head()

,N,cDNA alteration,Predicted protein alteration,Inheritance,Postaxial PD,Preaxial PD,Broad thumbs or halluces,Syndactyly,Macrocephaly,Widely spaced eyes,MRI Findings,Developmental delay,Additional findings
0,G029,327del,Phe109Leufs*50,F,–,FB,,+,–,–,–,–,"Precocious puberty, scaphocephaly"
1,G070,427G>T,Glu143*,F,HB,FL,,–,+,,,–,
2,Mother,427G>T,Glu143*,F,–,–,,–,–,–,,–,
3,G118,444C>A,Tyr148*,F,–,FB,BT,+,+,,,–,
4,G13684,444C>A,Tyr148*,F,–,FB,,+,+,+,,–,


In [44]:
column_mapper_d = defaultdict(ColumnMapper)

In [45]:
postaxial_d = {'HB': 'Postaxial hand polydactyly',
              'FB': 'Postaxial foot polydactyly',}
postaxialMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=postaxial_d, 
                                     negative_symbol="–", negative_label='Postaxial polydactyly')
postaxialMapper.preview_column(df1["Postaxial PD"])
column_mapper_d["Postaxial PD"] = postaxialMapper

In [46]:
preaxial_d = {'HB': 'Preaxial hand polydactyly',
              'FB': 'Preaxial foot polydactyly',
              'FL': 'Preaxial foot polydactyly',}
preaxialMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=preaxial_d,
                                   negative_symbol="–", negative_label='Preaxial polydactyly')
preaxialMapper.preview_column(df1["Preaxial PD"])
column_mapper_d["Preaxial PD"] = preaxialMapper

In [47]:
thumb_d = {"BT": "Broad thumb", 
          "BH": "Broad hallux",
          "+": [ "Broad thumb", "Broad hallux"]}
thumbMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=thumb_d,
                                negative_symbol="–", negative_label="Broad thumb")
thumbMapper.preview_column(df1["Broad thumbs or halluces"])
column_mapper_d["Broad thumbs or halluces"] = thumbMapper

In [48]:
syndMapper = SimpleColumnMapper(hpo_id="HP:0001159", hpo_label="Syndactyly", observed="+", excluded="–")
syndMapper.preview_column(df1["Syndactyly"])
column_mapper_d["Syndactyly"] = syndMapper

In [49]:
macMapper = SimpleColumnMapper(hpo_id="HP:0000256", hpo_label="Macrocephaly", observed="+", excluded="–")
macMapper.preview_column(df1["Macrocephaly"])
column_mapper_d["Macrocephaly"] = macMapper

In [50]:
#Widely spaced eyes  Hypertelorism HP:0000316
htMapper = SimpleColumnMapper(hpo_id="HP:0000316", hpo_label="Hypertelorism", observed="+", excluded="–")
htMapper.preview_column(df1["Widely spaced eyes"])
column_mapper_d["Widely spaced eyes"] = htMapper

In [51]:
# MRI Findings
# CCA, corpus callosum agenesis; CCH, corpus callosum hypoplasia;
# pCCA, partial CCA;  VD, ventricular dilatation. #: Index cases are indicated in bold and related are below.
mri_d = {'CCH': 'Hypoplasia of the corpus callosum',
         'CCA': 'Agenesis of corpus callosum',
         'pCCA': 'Partial agenesis of the corpus callosum',
         'VD': 'Ventriculomegaly'}
mriMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=mri_d)
mriMapper.preview_column(df1["MRI Findings"])
column_mapper_d["MRI Findings"] = mriMapper

In [52]:
dd_d = {'+': 'Global developmental delay',
         'Mild': 'Mild global developmental delay',
         'Bilateral inguinal hernia': 'Inguinal hernia',
         'strabismus': 'Strabismus',
       "Cataract": "Cataract",
       "Seizures":"Seizure",
       "horseshoe kidney": "Horseshoe kidney",
       "Trigonocephaly": "Trigonocephaly",
       "macrosomia": "Macrosomia",
       "vermis dysgenesis": "Dysgenesis of the cerebellar vermis"}
ddMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=dd_d,
                              negative_symbol="–", negative_label="Global developmental delay")
ddMapper.preview_column(df1["Developmental delay"])
column_mapper_d["Developmental delay"] = ddMapper

In [53]:
df1["Additional findings"].unique()

array(['Precocious puberty, scaphocephaly', '\xa0', 'Delta phalanx',
       'Atrial septal defect', 'Umbilical hernia',
       'Bifid distal phalanx, BW= 4150', 'Cerebral prematurity sequelae',
       'Delta metacarpal, BW=4880', 'BW=4740',
       'Hypoplastic cerebellum, microretrognathism',
       'Bilateral keratoconus, umbilical and bilateral inguinal hernia',
       'Macrosomia', 'Neurofibromatosis type 1',
       'Brachydactyly, delta phalanx', 'Brachydactyly, speech delay',
       'Speech delay, exomphalos', 'Umbilical hernia, anterior anus',
       'Laryngomalacia', 'BW=4440', 'Supernumerary nipples'], dtype=object)

In [54]:
add_d = {'Precocious puberty': 'Precocious puberty',
         'scaphocephaly': 'Scaphocephaly',
         'Delta phalanx': 'Triangular shaped phalanges of the hand',
         'Bifid distal phalanx': 'Partial duplication of the distal phalanges of the hand',
       "Hypoplastic cerebellum": "Cerebellar hypoplasia",
       "microretrognathism":"Microretrognathia",
       "keratoconus": "Keratoconus",
       "umbilical": "Umbilical hernia",
         "Umbilical hernia": "Umbilical hernia",
       "inguinal hernia": "Inguinal hernia",
       "Macrosomia": "Large for gestational age",
        "Brachydactyly":"Brachydactyly",
         "anterior anus": "Anteriorly placed anus",
         "Laryngomalacia":"Laryngomalacia",
         "Supernumerary nipples": "Supernumerary nipple"
        }
addMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=add_d)
addMapper.preview_column(df1["Additional findings"])
column_mapper_d["Additional findings"] = addMapper

<h2>GLI3 Variants</h2>
<p>Variants are provided in table 1 according to NM_000168.6.</p>
<p>Note that the contents of the column "cDNA alteration" do not have the "c." required by HGVS, so we add it to all columns before proceding.</p>

In [55]:
df1['cDNA alteration'] = 'c.' + df1['cDNA alteration'].astype(str)

In [56]:
transcript='NM_000168.6'
genome = 'hg38'
varMapper = VariantColumnMapper(assembly=genome,
                                column_name='cDNA alteration', 
                                transcript=transcript, 
                                default_genotype='heterozygous')
varMapper.preview_column(df1['cDNA alteration'])

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.327del/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.327del: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.427G>T/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.427G>T: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.427G>T/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.427G>T: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.444C>A/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.444C>A: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.444

Not able to get variant for c.3559C>T: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.3950del/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.3950del: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.3950del/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.3950del: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.4099dup/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.4099dup: 'Variant' object has no attribute 'set_genotype'
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000168.6%3Ac.4324C>T/NM_000168.6?content-type=application%2Fjson
Not able to get variant for c.4324C>T: 'Variant' object has no attribute 'set_genoty

,variant
0,n/a
1,n/a
2,n/a
3,n/a
4,n/a
5,n/a
6,n/a
7,n/a
8,n/a
9,n/a
